<a href="https://colab.research.google.com/github/rexian/ML/blob/main/agenticai/crewai/customer_outreach_campaign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Key Elements of Tools:
* Versatility
* Fault Tolerance
* Caching

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
!pip install -U -q crewai crewai_tools langchain_community

In [19]:
from crewai import Agent, Task, Crew

In [20]:
import os
import getpass
os.environ['NVIDIA_NIM_API_KEY'] = getpass.getpass("Enter API key for NVIDIA AI Endpoints: ")

Enter API key for NVIDIA AI Endpoints: ··········


In [37]:
os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter OPENAI API Key: ")
os.environ['SERPER_API_KEY'] = getpass.getpass("Enter Serper API Key: ")

Enter OPENAI API Key: ··········
Enter Serper API Key: ··········


In [47]:
from crewai.llm import LLM

llm = LLM(
    model="nvidia_nim/meta/llama3-70b-instruct",
    temperature=0.7
)

## Creating Agents

In [48]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    llm = llm,
    verbose=True
)

In [49]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    llm = llm,
    verbose=True
)

## Creating Tools

In [50]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [51]:
directory_read_tool = DirectoryReadTool(directory='./nvidia')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Creating Custom Tool

* Every Tool needs to have a name and a description.
* For simplicity and classroom purposes, SentimentAnalysisTool will return positive for every text.
* When running locally, you can customize the code with your logic in the _run function.

In [52]:
from crewai.tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [53]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

In [54]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [55]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [56]:
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=True,
	  memory=False
)

## Running the Crew

In [57]:
inputs = {
    "lead_name": "NVIDIA",
    "industry": "Designing and manufacturing GPU, chipsets, multimedia software",
    "key_decision_maker": "Jensen Huang",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 39701250-a9ad-4ffc-affd-db85dcbdd69f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5fba4817-29e3-4aff-b556-93950d0b5ac7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5fba4817-29e3-4aff-b556-93950d0b5ac7
       Status: Executing Task...
    └── 🤖 Agent: Sales Representative
            Status: In Progress

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of NVIDIA, a company in the Designing and manufacturing GPU, chipsets, multimedia software sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


🤖 Agent: Sales Representative
    Status: In Progress

...



# Agent: Lead Sales Representative
## Final Answer: 
Here are three personalized email drafts tailored to NVIDIA, specifically targeting Jensen Huang:

Draft 1:
Subject: Congratulations on the Launch of DGX Spark and DGX Station!
Dear Jensen,
Congratulations on the recent launch of DGX Spark and DGX Station Personal AI Computers! This innovative technology is a significant milestone in the field of AI computing. I'm excited to explore how our solutions can support NVIDIA's AI and digital twin initiatives. Please let me know if you'd like to discuss potential collaborations.
Best regards, [Your Name]

Draft 2:
Subject: Accelerating AI Innovation with NVIDIA
Dear Jensen,
I'm thrilled to see NVIDIA's continued innovation in AI computing with the launch of DGX Spark and DGX Station. Our solutions align perfectly with your vision for AI-driven transformation. I'd love to schedule a call to discuss how we can support your initiatives and drive growth in various markets.
Best regards, [Your

🚀 Crew: crew
├── 📋 Task: 5fba4817-29e3-4aff-b556-93950d0b5ac7
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: 971bb2da-321a-49a2-8204-7b521a4f15cd
       Status: Executing Task...
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 5fba4817-29e3-4aff-b556-93950d0b5ac7
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: 971bb2da-321a-49a2-8204-7b521a4f15cd
       Assigned to: Lead Sales Representative
       Status: ✅ Completed
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 971bb2da-321a-49a2-8204-7b521a4f15cd                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 39701250-a9ad-4ffc-affd-db85dcbdd69f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Display final output

In [58]:
from IPython.display import Markdown
Markdown(result.raw)

Here are three personalized email drafts tailored to NVIDIA, specifically targeting Jensen Huang:

Draft 1:
Subject: Congratulations on the Launch of DGX Spark and DGX Station!
Dear Jensen,
Congratulations on the recent launch of DGX Spark and DGX Station Personal AI Computers! This innovative technology is a significant milestone in the field of AI computing. I'm excited to explore how our solutions can support NVIDIA's AI and digital twin initiatives. Please let me know if you'd like to discuss potential collaborations.
Best regards, [Your Name]

Draft 2:
Subject: Accelerating AI Innovation with NVIDIA
Dear Jensen,
I'm thrilled to see NVIDIA's continued innovation in AI computing with the launch of DGX Spark and DGX Station. Our solutions align perfectly with your vision for AI-driven transformation. I'd love to schedule a call to discuss how we can support your initiatives and drive growth in various markets.
Best regards, [Your Name]

Draft 3:
Subject: Unlocking the Power of AI with NVIDIA
Dear Jensen,
The recent launch of DGX Spark and DGX Station is a testament to NVIDIA's commitment to AI innovation. I believe our solutions can complement your efforts in accelerating AI adoption across industries. Let's explore potential areas of collaboration and drive meaningful impact together.
Best regards, [Your Name]